In [13]:
import numpy as np

In [14]:
train_data = np.random.randint(0, 100, size=(10, 215, 215, 1))
train_labels = np.eye(2)[np.random.randint(0, 2, size=(10, 1)).reshape(-1)]
people = [2, 2, 3, 3, 4, 4, 6, 7, 9 , 0]


In [65]:
def split_clients_into_dict(people, *args):
    array = []
    for arg in args:
        dic = {}
        for key, value in zip(people, arg):
            dic.setdefault(key, []).append(value)
        for key in dic.keys():
            dic[key] = np.array(dic[key])
        if len(args) == 1:
            return dic
        else:
            array.append(dic)
    return tuple(array)

In [70]:
td, tl = split_clients_into_dict(people, train_data, train_labels)
# train_data = split_clients_into_dict(people, train_data)

In [72]:
tl

{2: array([[0., 1.],
        [0., 1.]]), 3: array([[1., 0.],
        [1., 0.]]), 4: array([[1., 0.],
        [0., 1.]]), 6: array([[1., 0.]]), 7: array([[1., 0.]]), 9: array([[1., 0.]]), 0: array([[0., 1.]])}